# Neo4j Hello World (Notebook)

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it into a pandas DataFrame.

**Assumes** 
 
 
- Neo4j service is running at `bolt://localhost:${URI_PORT}` with the user and password set in the `.env` file. **Run `docker compose up -d`**.
- Ollama service is up on `http://localhost:11434` (ollama default). **Run `ollama serve` and pull the model `ollama pull nomic-embed-text`** (if not pulled yet).



In [110]:
import os
from dotenv import load_dotenv  
import yaml
from pathlib import Path
from pprint import pprint
from termcolor import cprint
import ollama
import requests
from typing import Literal

from langchain_neo4j import Neo4jGraph


In [111]:
load_dotenv()  # Load local environment variables

URI = "bolt://localhost:" + os.environ.get("URI_PORT")
NEO4J_USER = os.environ.get("NEO4J_USER")
NEO4J_PWD = os.environ.get("NEO4J_PASSWORD")
NEO4J_DB = os.getenv("NEO4J_DATABASE", "neo4j")    # 👈 choose DB here
EMBED_MODEL = "nomic-embed-text:latest"

cprint(f"Connecting to Neo4j at {URI} with user {NEO4J_USER} and password {NEO4J_PWD}", "green")

Connecting to Neo4j at bolt://localhost:7687 with user neo4j and password test1234


In [112]:
# load cypher queries from yaml file
queries = yaml.safe_load(Path("queries.yaml").read_text())
queries.keys()  # list available queries

dict_keys(['constraints', 'create_seed', 'show_people', 'show_companies', 'match_adjacency', 'show_info', 'add_info', 'create_vector_indexes', 'delete_all'])

## Using Langchain wrapper for Neo4j

In [ ]:
kg = Neo4jGraph(url=URI, username=NEO4J_USER, password=NEO4J_PWD, database=NEO4J_DB)

### 1. Create sample data

<p align="center">
  <img src="media/KG_step1_populate_graph.svg" width="550">
</p>


- **Entities**: Person and Company nodes with unique constraints (unique name and uuid)
- **Relationships**: KNOWS (person-to-person) and WORKS_AT (person-to-company)
- **Properties**: Basic attributes (name, age, education, industry)


In [ ]:
# KG

wipe_at_init = True # delete everything at the start 

# Interact directly with KG, no need for driver context.

cprint(f"\n== Connected to Neo4j database: {NEO4J_DB}", "green")

cprint("\n== Creating constraints (if not exist)", "green")
for q in queries["constraints"]:
    kg.query(q)
print(" ok")

cprint("\n== Init Cleanup.", "green")
if wipe_at_init:
    for q in queries["delete_all"]:
        kg.query(q)
    print(" ok")
else:
    print(" skipped")
    
cprint("\n== Creating sample data", "green")
kg.query(queries["create_seed"])
print(" ok")

cprint("\n== Query: list all people", "green")
records = kg.query(queries["show_people"]) # <class 'list'>
for r in records:
    print(r)
    
cprint("\n== Query: list all companies", "green")
records = kg.query(queries["show_companies"]) # <class 'list'>
for r in records:
    print(r)

cprint("\n== Query: adjacency (who knows whom)", "green")
records = kg.query(queries["match_adjacency"]) # <class 'list'>
for r in records:
    print(r)


== Connected to Neo4j database: neo4j

== Creating constraints (if not exist)
 ok

== Init Cleanup.
 ok

== Creating sample data
 ok

== Query: list all people
{'name': 'Paula', 'age': 25, 'education': 'Computer Engineering'}
{'name': 'Guillermo', 'age': 26, 'education': 'Industrial Engineering'}
{'name': 'Gabriela', 'age': 26, 'education': 'Physics'}
{'name': 'Iria', 'age': 27, 'education': 'Physics'}
{'name': 'Cristina', 'age': 27, 'education': 'Physics'}

== Query: list all companies
{'name': 'Indra', 'industry': 'Engineering'}
{'name': 'CIEMAT', 'industry': 'Scientific Research'}
{'name': 'CBM', 'industry': 'Scientific Research'}

== Query: adjacency (who knows whom)
{'person': 'Cristina', 'knows': ['Gabriela', 'Iria'], 'works_at': 'CBM'}
{'person': 'Gabriela', 'knows': ['Cristina', 'Iria'], 'works_at': 'CIEMAT'}
{'person': 'Guillermo', 'knows': ['Paula', 'Iria'], 'works_at': 'Indra'}
{'person': 'Iria', 'knows': ['Cristina', 'Paula', 'Gabriela', 'Guillermo'], 'works_at': 'Indra'}


### 2. Add rich text info

<p align="center">
  <img src="media/KG_step2_generate_rich_descriptions.svg" width="750">
</p>


In [115]:
cprint("\n== Query: Adding descriptions, appearance and summaries", "green")
for q in queries["add_info"]:
    kg.query(q)
print(" ok")

records = kg.query(queries["show_info"])
for r in records:
    print(r)



== Query: Adding descriptions, appearance and summaries
 ok
{'labels(n)': ['Person'], 'n.name': 'Iria', 'n.info': 'Iria is 27 years old and studied Physics.Iria has blue eyes and long brunette and wavy hair. She likes to paint her nails in red or purple colours. She usually wears long earrings.'}
{'labels(n)': ['Person'], 'n.name': 'Guillermo', 'n.info': 'Guillermo is 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.'}
{'labels(n)': ['Person'], 'n.name': 'Gabriela', 'n.info': "Gabriela is 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-style clothes."}
{'labels(n)': ['Person'], 'n.name': 'Paula', 'n.info': 'Paula is 25 years old and studied Computer Engineering.Paula short hair in a wolfcut style. She wears long and wide pants and sneakers to the laboratory.'}
{'labels(n)': ['Person'

### 3. Create property embeddings (first step into RAG) 

<p align="center">
  <img src="media/KG_step3_generate_property_embeddings.svg" width="750">
</p>

**RAG** implementation requires selecting a **property to embed and use for similarity searches**. 

Description properties containing **rich text** work well for this purpose, as they provide richer semantic information. In our example, we'll use *info* and *info*.

In order to do so, we create two vector indexes in Neo4j:

- **Vector index *person_node_info_idx***: based on property ***info_emb*** for nodes of type "Person"
-  **Vector index *company_node_info_idx***: based on property ***info_emb*** for nodes of type "Company"

After that, we create the embeddings (this happens for both Person nodes and Company nodes):

- **Property *info*** ---`nomic-embed-text`---> **Property *info_emb***


In [116]:
def embed_property(element: Literal["node", "relationship"],type_name:str, property_name:str):
        
    if element == "node":
        cprint(f"\nGenerating embeddings for (n:{type_name}) on n.{property_name}", "green")
        records = list(kg.query(f"""
            MATCH (n:{type_name})
            WHERE n.{property_name} IS NOT NULL AND n.{property_name} <> ''
            AND n.{property_name}_emb IS NULL
            RETURN n.uuid AS uuid, n.{property_name} AS txt
            """))
        for r in records:
            vec = ollama.embed(model="nomic-embed-text", input=r["txt"])["embeddings"][0]
            kg.query(
                f"""
                MATCH (n:{type_name} {{uuid: $uuid}})
                SET n.{property_name}_emb = $vec
                """,
                params={"uuid": r["uuid"], "vec": vec},
            )
            print(f"  text: {r['txt']}\n  vec: {vec[:3]}")
            
    elif element == "relationship":
        
        cprint(f"\nGenerating embeddings for [r:{type_name}] on r.{property_name}", "green")
        records = list(kg.query(f"""
            MATCH ()-[r:{type_name}]-()
            WHERE r.{property_name} IS NOT NULL AND r.{property_name} <> ''
            AND r.{property_name}_emb IS NULL
            RETURN r.uuid AS uuid, r.{property_name} AS txt
            """))
        for r in records:
            vec = ollama.embed(model="nomic-embed-text", input=r["txt"])["embeddings"][0]
            kg.query(
                f"""
                MATCH ()-[r:{type_name} {{uuid: $uuid}}]-()
                SET r.{property_name}_emb = $vec
                """,
                params={"uuid": r["uuid"], "vec": vec},
            )
            print(f"  text: {r['txt']}\n  vec: {vec[:3]}")
            
# Create vector indexes (once)

for q in queries["create_vector_indexes"]:
    kg.query(q)

# Show created vector indexes
results = kg.query("SHOW VECTOR INDEXES")
idx = list(results)
cprint(f"\nFound {len(idx)} vector index entries.", "green")
for r in idx:
    cprint("-"*20,"green")
    pprint(r)


Found 3 vector index entries.
--------------------
{'entityType': 'NODE',
 'id': 11,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Company'],
 'lastRead': None,
 'name': 'company_node_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['info_emb'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'RELATIONSHIP',
 'id': 12,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['KNOWS'],
 'lastRead': None,
 'name': 'knows_relationship_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['info_emb'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}
--------------------
{'entityType': 'NODE',
 'id': 16,
 'indexProvider': 'vector-2.0',
 'labelsOrTypes': ['Person'],
 'lastRead': None,
 'name': 'person_node_info_idx',
 'owningConstraint': None,
 'populationPercent': 100.0,
 'properties': ['info_emb'],
 'readCount': None,
 'state': 'ONLINE',
 'type': 'VECTOR'}


In [117]:
# (p:PERSON): create embeddings only for nodes missing them
embed_property("node", "Person", "info")

# (c:COMPANY): create embeddings only for nodes missing them
embed_property("node", "Company", "info")

# [r:KNOWS]: create embeddings only for nodes missing them
embed_property("relationship", "KNOWS", "info")


Generating embeddings for (n:Person) on n.info
  text: Iria is 27 years old and studied Physics.Iria has blue eyes and long brunette and wavy hair. She likes to paint her nails in red or purple colours. She usually wears long earrings.
  vec: [0.021614064, 0.025623962, -0.15588778]
  text: Guillermo is 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.
  vec: [-0.025044667, 0.046419382, -0.15375015]
  text: Gabriela is 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-style clothes.
  vec: [0.026251283, 0.022286754, -0.17932063]
  text: Paula is 25 years old and studied Computer Engineering.Paula short hair in a wolfcut style. She wears long and wide pants and sneakers to the laboratory.
  vec: [0.0076446, 0.035536703, -0.16631141]
  text: Cristina is 27 years old and studied Physics.Cr

### 4. Search 

Whenever we query this graph, we can use two different but complementary search techniques:

1. **KG Retreival**: through **Neo4J Cypher Query Language (CQL)** we can query precise entities and relations. The input query must be translated into CQL to get the desired results.

2. **Vector Retrieval**: **embedding the input query**, we can make a vector search against the vector indexes defined above.

The results will be a combination of both searches.

<p align="center">
  <img src="media/KGRAG_schema.svg">
</p>


In [ ]:
# From query/question to cypher query language (cql) TODO
def create_question_cql(question:str):
    cql_query = ""
    #cql_query = "MATCH (person)-[:KNOWS]-(:Person {name:'Cristina'})" 
    return cql_query

In [119]:
# From user query/question to question embedding
def create_question_embedding(question:str):
    cprint(f"\nGenerating embeddings for question '{question}'", "green")
    vec = ollama.embed(model="nomic-embed-text", input=question)["embeddings"][0] 
    print(f"  text: {question}\n  vec: {vec[:10]}\n")
    return vec

In [120]:
# Query Nodes

question = "Who shaved its head this summer?"
question_emb = create_question_embedding(question)
question_cql = create_question_cql(question)

index_name = "person_node_info_idx"
k = 5

res = kg.query(
    f"""
    CALL db.index.vector.queryNodes($index_name, $k, $question_emb)
    YIELD node, score
    {question_cql}
    RETURN score, node {{.* , info_emb: ""}}
    ORDER BY score DESC
    """,
    params = {"index_name": index_name, "k": k, "question_emb": question_emb},
)
for r in res:
    print(r)
    
    
# ---

question = "Which company investigates Cancer?"
question_emb = create_question_embedding(question)
question_cql = create_question_cql(question)

index_name = "company_node_info_idx"
k = 5

res = kg.query(
    f"""
    CALL db.index.vector.queryNodes($index_name, $k, $question_emb)
    YIELD node, score
    {question_cql}
    RETURN score, node {{.* , info_emb: ""}}
    ORDER BY score DESC
    """,
    params = {"index_name": index_name, "k": k,"question_emb": question_emb},
)

for r in res:
    print(r)
    
    
# Query Relationships 

question = "Who is helping Iria at work?"
question_emb = create_question_embedding(question)
question_cql = create_question_cql(question)

index_name = "knows_relationship_info_idx"
k = 5

res = kg.query(
    f"""
    CALL db.index.vector.queryRelationships($index_name, $k, $question_emb)
    YIELD relationship, score
    {question_cql}
    RETURN score, relationship {{.* , info_emb: ""}}
    ORDER BY score DESC
    """,
    params = {"index_name": index_name, "k": k,"question_emb": question_emb},
)

for r in res:
    print(r)
        


Generating embeddings for question 'Who shaved its head this summer?'


  text: Who shaved its head this summer?
  vec: [0.022573026, -0.015476549, -0.17212495, 0.0018683294, -0.038688686, 0.044817436, 0.01842398, 0.013745231, 0.050375726, 0.008345599]

{'score': 0.8364357948303223, 'node': {'name': 'Guillermo', 'age': 26, 'uuid': 'b1c9afbd-3453-434a-b299-857a14476aeb', 'info_emb': '', 'education': 'Industrial Engineering', 'info': 'Guillermo is 26 years old and studied Industrial Engineering.Guillermo has brown eyes and short hair. He has a very fancy shirt that he takes to all important events. He shaved his head this summer.'}}
{'score': 0.7836265563964844, 'node': {'name': 'Gabriela', 'age': 26, 'uuid': 'c930ee04-a50f-46b2-a01e-f1a21ea536bf', 'info_emb': '', 'education': 'Physics', 'info': "Gabriela is 26 years old and studied Physics.Gabriela has long curly hair with babylights. She's petite and likes to wear hippie-style clothes."}}
{'score': 0.7775712013244629, 'node': {'name': 'Cristina', 'age': 27, 'uuid': 'b2eb582c-01be-4d1e-825d-dd61fda1bc9f', '